In [125]:
import pandas as pd
import uuid
from sklearn.impute import KNNImputer
from countryinfo import CountryInfo
import pycountry_convert as pc
import warnings
import json
warnings.simplefilter(action='ignore', category=FutureWarning)
with open('keys.json') as json_file:
    column_keys = json.load(json_file)
data = pd.read_csv('./data/HNP_StatsData.csv')
countries_info = pd.read_csv('./data/HNP_StatsCountry.csv')
disaster_data = pd.read_csv('./data/emdat_public_2022_01_31.csv')


/tmp/ipykernel_132891/3842362895.py:13: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  disaster_data = pd.read_csv('./data/emdat_public_2022_01_31.csv')


In [126]:
countries = [
  'CAN',
  'USA',
  'MEX',
  'IND',
  'CHN',
  'SDN',
  'BGD',
  'BRA',
  'NER'
]
countries_full = [
  'Canada',
  'United States of America (the)',
  'Mexico',
  'India',
  'China',
  'Sudan',
  'Bangladesh',
  'Brazil',
  'Niger'
]

education_keys = [
  'SE.ADT.LITR.ZS',
  'SE.ADT.LITR.FE.ZS',
  'SE.ADT.LITR.MA.ZS',
  'SE.PRM.ENRR',
  'SE.PRM.ENRR.FE',
  'SE.PRM.ENRR.MA',
  'SE.SEC.ENRR',
  'SE.SEC.ENRR.FE',
  'SE.SEC.ENRR.MA',
  'SE.XPD.TOTL.GD.ZS',
  'SE.TER.ENRR',
  'SE.PRM.CMPT.ZS'
]

health_keys = [
  'SH.DTH.COMM.ZS',
  'SH.DTH.NCOM.ZS',
  'SH.XPD.CHEX.GD.ZS',
  'SH.MED.BEDS.ZS',
  'SH.IMM.MEAS',
  'SH.STA.OWGH.ME.ZS',
  'SH.STA.OWGH.ME.ZS',
  'SH.TBS.INCD',
  'SH.ANM.CHLD.ZS',
  'SH.IMM.POL3',
  'SH.IMM.IDPT',
  'SH.STA.DIAB.ZS',
  'SH.UHC.SRVS.CV.XD',
  'SH.MED.NUMW.P3'
]

quality_of_life_keys = [
  'SH.STA.BASS.ZS',
  'SH.STA.SMSS.ZS',
  'SH.STA.WASH.P5',
  'SH.H2O.BASW.ZS',
  'SH.H2O.SMDW.ZS',
  'SL.TLF.TOTL.IN',
  'SL.UEM.TOTL.MA.ZS',
  'SL.UEM.TOTL.FE.ZS',
  'SH.MMR.WAGE.ZS',
  'SI.POV.NAHC',
  'SH.STA.ODFC.ZS',
  'SL.TLF.TOTL.FE.ZS',
  'SH.STA.AIRP.P5',
  'SH.STA.BRTC.ZS ',
]

population_keys = [
  'SP.POP.TOTL',
  'SP.DYN.LE00.MA.IN',
  'SP.DYN.LE00.FE.IN ',
  'SP.POP.GROW ',
  'SP.DYN.LE00.IN',
  'SM.POP.NETM',
  'SP.RUR.TOTL',
  'SP.RUR.TOTL.ZG',
  'SI.POV.RUHC',
  'SP.URB.TOTL',
  'SP.URB.GROW',
  'SI.POV.URHC',
  'SP.POP.TOTL.MA.ZS',
  'SP.POP.TOTL.FE.ZS'
]

columns = [
  'Country Name',
  'Country Code',
  'Indicator Name',
  'Indicator Code',
  '2005',
  '2006',
  '2007',
  '2008',
  '2009',
  '2010',
  '2011',
  '2012',
  '2013',
  '2014',
  '2015',
  '2016',
  '2017',
  '2018',
  '2019',
  '2020'
]

country_columns = [
  "Long Name",
  "Short Name",
  "Country Code",
  "Region",
  # Continent
  "Currency Unit",
  # Capital
  # Number of languages
  # area
  "Income Group",
  "Latest population census"  
]
disaster_columns = [
  'Dis No',
  'Country',
  'Start Year',
  'Start Month',
  'Start Day',
  'End Year',
  'End Month',
  'End Day',
  'Disaster Group',
  'Disaster Subgroup',
  'Disaster Type',
  'Disaster Subtype',
  'Total Affected',
  'Total Deaths',  
]

## Generate Seed Data for World Bank Data
This code block generates csv files to be used to populate the Data Mart with data directly fetched from the wolrd bank source file, namely Education, Health, Population, and Quality of Life

In [127]:
def getData(columns, keys, countries, tableName):
  selected_data = pd.DataFrame()
  selected_data_imputed = pd.DataFrame()
  for country in countries:
    current_info = data.loc[data['Country Code'] == country]
    current_info = current_info.loc[data['Indicator Code'].isin(keys)]
    current_info = current_info[columns]
    cols = current_info.columns.tolist()
    # convert Year into its own column

    if tableName =='Population':
      print(current_info.head())
   
    current_info = current_info.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name= "Year", value_name="Value")
    cols = current_info.columns.tolist()
    cols.insert(2, cols.pop(cols.index("Year")))
    current_info = current_info.reindex(columns= cols) #Reposition the Year column into index 2
    current_info = current_info.drop('Indicator Code', 1) #Drop the Indicator Code column
    current_info = current_info.pivot_table('Value', ['Country Name', 'Country Code', 'Year'], 'Indicator Name').reset_index() #Pivot the table so that Each value in indicator Name is its own column
    cols = current_info.columns.tolist()
    numeric_info = current_info.iloc[:,3:]
    imputer = KNNImputer()
    imputer.fit(numeric_info)
    sample_incomplete_rows = numeric_info[numeric_info.isnull().any(axis=1)].head()
    input_x = imputer.transform(numeric_info)
    imputed_data = pd.DataFrame(input_x, columns=numeric_info.columns, index=numeric_info.index)
    result = pd.concat([current_info.iloc[:,:3], imputed_data], axis=1, join="inner")
    selected_data_imputed = pd.concat((selected_data_imputed, result[cols]))
    selected_data = pd.concat((selected_data, current_info))


  selected_data_imputed.insert(0, 'key', [i for i in range(len(selected_data_imputed.index))])
  selected_data.insert(0, 'key', [i for i in range(len(selected_data.index))])
  # print(selected_data_imputed.head())
  selected_data_imputed = selected_data_imputed.rename(columns=column_keys)
  selected_data_imputed.to_csv(path_or_buf=f'./seed_data/{tableName}_seed.csv', index=False)

  # selected_data.to_csv(path_or_buf=f'./seed_data/{tableName}_Unimputed.csv', columns=cols, index=False)
tables = {
  'Education': education_keys,
  'Health' : health_keys,
  'Quality_of_life' : quality_of_life_keys,
  'Population': population_keys
  }
for key in tables:
  getData(columns, tables[key], countries, key)

      Country Name Country Code                              Indicator Name  \
38537       Canada          CAN      Life expectancy at birth, male (years)   
38538       Canada          CAN     Life expectancy at birth, total (years)   
38575       Canada          CAN                               Net migration   
38720       Canada          CAN  Population, female (% of total population)   
38722       Canada          CAN    Population, male (% of total population)   

          Indicator Code       2005       2006          2007       2008  \
38537  SP.DYN.LE00.MA.IN  77.900000  78.100000  7.830000e+01  78.500000   
38538     SP.DYN.LE00.IN  80.192683  80.343902  8.054390e+01  80.695122   
38575        SM.POP.NETM        NaN        NaN  1.326431e+06        NaN   
38720  SP.POP.TOTL.FE.ZS  50.425044  50.415091  5.040446e+01  50.394821   
38722  SP.POP.TOTL.MA.ZS  49.574956  49.584909  4.959554e+01  49.605179   

            2009       2010       2011          2012       2013       2014

## Generate Seed Data for Countries
This Code block generates the csv files used to populate Country table in the data mart

In [128]:
selected_data = pd.DataFrame()
for country in countries:
  current_info = countries_info.loc[countries_info['Country Code'] == country][country_columns]
  alpha =countries_info.loc[countries_info['Country Code'] == country]["2-alpha code"].values[0]
  continent = pc.country_alpha2_to_continent_code(alpha)
  current_countryInfo = CountryInfo(alpha)
  capital = current_countryInfo.capital()
  area = current_countryInfo.area()
  numLanguages = len(current_countryInfo.languages())
  current_info.insert(len(current_info.columns),'continent', continent)
  current_info.insert(len(current_info.columns),'capital', capital)
  current_info.insert(len(current_info.columns),'area', area)
  current_info.insert(len(current_info.columns),'num_languages', numLanguages)
  selected_data = pd.concat((selected_data, current_info))

selected_data.insert(0, 'key', [i for i in range(len(selected_data.index))])
selected_data = selected_data.rename(columns=column_keys)
selected_data = selected_data.replace('2020 (expected)','2020')
selected_data.to_csv(path_or_buf=f'./seed_data/countries_seed.csv', columns=selected_data.columns, index=False) 

## Generate Seed Data for Events
The events table includes data from two sources, one source includes infromation on disasters, natural and otherwise, the other includes informaiton on terrorist attacks. The information from these two data sources has been conformed to fit into the event dimension and the CSV file to populate this dimension is generated using the code below

In [129]:
selected_data = pd.DataFrame(columns = disaster_columns)
for country in countries_full:
    current_info = disaster_data.loc[disaster_data['Country'] == country]
    current_info = current_info.loc[disaster_data['Start Year'].between(2005, 2020)]
    selected_data = selected_data.append(current_info)
# selected_data['Total Deaths'] = np.where((selected_data['Total Affected'] > 0) &  (selected_data['Total Deaths'].isna()), 0 , "" )
#Fills in missing values in Total affected and Total deaths based off assumptions.
selected_data.loc [(selected_data['Total Affected'] > 0) & (selected_data['Total Deaths'].isnull()), 'Total Deaths'] = 0
selected_data.loc [(selected_data['Total Affected'].isnull()) & (selected_data['Total Deaths'].isnull()), 'Total Deaths'] = 0 
selected_data.loc [(selected_data['Total Affected'].isnull()) & (selected_data['Total Deaths'].notnull()), 'Total Affected'] = selected_data['Total Deaths']
selected_data.loc [(selected_data['Disaster Subtype'].isnull()), 'Disaster Subtype'] = "None"
selected_data = selected_data.replace('United States of America (the)','United States of America')

#Deletes columns with missing data in Start day
indexs = selected_data[selected_data['Start Day'].isnull()].index
selected_data.drop(indexs, inplace=True)
selected_data = selected_data.rename(columns=column_keys)
#
# Add Lilian's code to get  terrorist attacks here, and add to dataframe!
#
selected_data = selected_data.drop(columns=['key'])
selected_data.insert(0, 'key', [i for i in range(len(selected_data.index))])
print(selected_data.head())

selected_data.to_csv(path_or_buf=f'./seed_data/Events_seed.csv', index=False)

       key Country  start_year  start_month  start_day  end_year  end_month  \
14009    0  Canada        2005          9.0       27.0      2005        9.0   
14010    1  Canada        2005          6.0        7.0      2005        7.0   
14013    2  Canada        2005         11.0       30.0      2005       12.0   
14014    3  Canada        2005          5.0        8.0      2005        5.0   
14015    4  Canada        2005          9.0       26.0      2005        9.0   

       end_day disaster_group disaster_subgroup  ...  \
14009     29.0        Natural      Hydrological  ...   
14010      1.0        Natural      Hydrological  ...   
14013      1.0  Technological     Technological  ...   
14014      8.0  Technological     Technological  ...   
14015     29.0        Natural    Meteorological  ...   

      Reconstruction Costs, Adjusted ('000 US$) Insured Damages ('000 US$)  \
14009                                       NaN                        NaN   
14010                           

## Date Seed Data
Here entries for the  date table are generated, this is simply done by looping through 12 months for each year in the perisod spedified in the project outline

In [130]:
months = [
  'January',
  'February',
  'March',
  'April',
  'May',
  'June',
  'July',
  'August',
  'September',
  'October',
  'November',
  'December'
]
df = pd.DataFrame(columns=['key', 'name', 'year', 'month_number', 'quarter', 'decade'])
key = 0
for year in range(2005, 2021):
  for month in range(12):
    df = df.append({
      'key': key,
      'name': months[month],
      'year': year,
      'month_number': month,
      'quarter': month//3,
      'decade': year//10,
    }, ignore_index=True)
    key +=1

df.to_csv(path_or_buf=f'./seed_data/date_seed.csv', index=False)


## Fact Table Generate

Here, the entries for the fact table are generated

In [131]:
dates = pd.read_csv('./seed_data/date_seed.csv')
events = pd.read_csv('./seed_data/Events_seed.csv')
education = pd.read_csv('./seed_data/Education_seed.csv')
health = pd.read_csv('./seed_data/Health_seed.csv')
population = pd.read_csv('./seed_data/Population_seed.csv')
qualityOfLife = pd.read_csv('./seed_data/Quality_of_life_seed.csv')
countries = pd.read_csv('./seed_data/countries_seed.csv')

fact_columns = [
  'key',
  'country_key',
  'date_key',
  'education_key',
  'health_key',
  'quality_of_life_key',
  'event_key',
]

def extractKey(key):
  if len(key) ==0:
    return None
  else:
    return key[0]

def checkDate(start_year, start_month, end_year, end_month, current_year, current_month):
  if start_year == end_year == current_year:
    if current_month >= start_month and current_month <= end_month:
      return True
  elif start_year == current_year:
    if current_month >= start_month:
      return True
  elif end_year == current_year:
    if current_month <= end_month:
      return True
  elif current_year > start_year and current_year < end_year:
    return True
  return False

In [132]:
facts = pd.DataFrame( )
index = 0
for date_index in dates.index:
  for country_index in countries.index:
    country_code = countries['alpha_code'][country_index]
    country_name = countries['short_name'][country_index]
    year = dates['year'][date_index]
    month = dates['month_number'][date_index]
    education_key = extractKey(education[(education['alpha_code']==country_code) & (education['Year']==year)]['key'].values)
    health_key = extractKey(health[(health['alpha_code']==country_code) & (health['Year']==year)]['key'].values)
    quality_of_life_key = extractKey(qualityOfLife[(qualityOfLife['alpha_code']==country_code) & (qualityOfLife['Year']==year)]['key'].values)
    population_key = extractKey(population[(population['alpha_code']==country_code) & (population['Year']==year)]['key'].values)
    # print(country_name)
    country_events = events[events['Country']==country_name]
    current_events = []
    for event_index in country_events.index:
      if checkDate(country_events['start_year'][event_index],country_events['start_month'][event_index],country_events['end_year'][event_index],country_events['end_month'][event_index], year, month):
        current_events.append(country_events['key'][event_index])
    for event_key in current_events:
      facts = facts.append(pd.DataFrame([[index, country_index, date_index,education_key, health_key,quality_of_life_key,event_key]], columns=fact_columns))
      index += 1
print(facts.head())
print(fact_columns)
facts.to_csv(path_or_buf=f'./seed_data/Facts.csv', index=False) 
      #   selected_data_imputed.insert(0, 'key', [i for i in range(len(selected_data_imputed.index))])

# 
# 

   key  country_key  date_key  education_key  health_key  quality_of_life_key  \
0    0            2         1             32          32                   32   
0    1            3         1             48          48                   48   
0    2            3         1             48          48                   48   
0    3            4         1             64          64                   64   
0    4            4         1             64          64                   64   

   event_key  
0        458  
0        628  
0        630  
0       1115  
0       1117  
['key', 'country_key', 'date_key', 'education_key', 'health_key', 'quality_of_life_key', 'event_key']


## Dropping Unnecessary Data
Now that the fact table has been created, the dimensional tables no longer need to store the country or date information, so we will now go through each of those files and remove those columns

In [133]:
education = pd.read_csv('./seed_data/Education_seed.csv')
health = pd.read_csv('./seed_data/Health_seed.csv')
population = pd.read_csv('./seed_data/Population_seed.csv')
qualityOfLife = pd.read_csv('./seed_data/Quality_of_life_seed.csv')

tables = {
  'Education': education,
  'Health': health,
  'Population': population,
  'Quality_of_life': qualityOfLife,
}

for key in tables:
  tmp = tables[key]
  tmp = tmp.drop(columns=['country_name', 'alpha_code', 'Year'])
  tmp.to_csv(path_or_buf=f'./seed_data/{key}_seed.csv', index=False)

